In [25]:
import pandas as pd, altair as alt, json, os
from datetime import datetime
import sys, importlib

# Append absolute path to folder holding api_wrapper
absolute_path_to_api_wrapper = "/Users/joshhellings/Documents/ECO/RADataHub/ChartOfTheDay/templates/"
sys.path.append(absolute_path_to_api_wrapper)
import api_wrapper.api_hub as api_hub
import altair_wrapper.eco_styles as styles

---

### UK Debt

ONS - [public sector finances](https://www.ons.gov.uk/economy/governmentpublicsectorandtaxes/publicsectorfinance/bulletins/publicsectorfinances/october2023)
    - November 21st 2023 release

In [5]:
api = api_hub.EconDataAPI()

- Total public sector spending - `KX5Q`
- Total public sector receipts - `JW2O`

Source dataset:
- Public sector finances time series - `PUSF`

In [11]:
spend = api.get_ons_data(dataset_id='PUSF', series_id='KX5Q')
reven = api.get_ons_data(dataset_id='PUSF', series_id='JW2O')

In [20]:
print(spend['date'].iloc[0].strftime('%Y-%m-%d'))
print(spend['date'].iloc[-1].strftime('%Y-%m-%d'))
spend.tail()

1997-04-01
2023-10-01


,date,month,value,year
314,2023-06-01,June,103946.0,2023
315,2023-07-01,July,97547.0,2023
316,2023-08-01,August,94662.0,2023
317,2023-09-01,September,98365.0,2023
318,2023-10-01,October,99833.0,2023


In [21]:
print(reven['date'].iloc[0].strftime('%Y-%m-%d'))
print(reven['date'].iloc[-1].strftime('%Y-%m-%d'))
reven.tail()

1997-04-01
2023-10-01


,date,month,value,year
314,2023-06-01,June,84184.0,2023
315,2023-07-01,July,94783.0,2023
316,2023-08-01,August,83569.0,2023
317,2023-09-01,September,83723.0,2023
318,2023-10-01,October,84937.0,2023


##### Merge dataframes

In [53]:
spend['series'] = 'Spending'
reven['series'] = 'Receipts'

df_dual = pd.concat([spend, reven])

Let's also download the debt % of GDP data from the ECO API

In [32]:
# ECO API has 12-month inflation, we want 1-month updates
url = 'https://api.economicsobservatory.com/gbr/debt?vega'

# load data from ECO API in pandas dataframe
debt_gdp = pd.read_json(url)

debt_gdp.tail()

,date,value
363,2023-06-01,97.3
364,2023-07-01,96.4
365,2023-08-01,96.5
366,2023-09-01,96.4
367,2023-10-01,97.8


Let's also create a new dataframe for public sector borrowing and debt % of gdp. 
- borrowing = spending - receipts
- debt % of gdp, from Eco Api

In [70]:
# filter `debt_gdp` to match time period of `spend` and `reven`
debt_gdp = debt_gdp[(debt_gdp['date'] >= spend['date'].iloc[0]) & (debt_gdp['date'] <= spend['date'].iloc[-1])].reset_index(drop=True)

# Create new dataframe
df = pd.DataFrame({'date': spend['date'], 'year': spend['date'].dt.year, 'borrowing': -(reven['value'] - spend['value']), 'debt': debt_gdp['value']})

In [71]:
df

,date,year,borrowing,debt
0,1997-04-01,1997,2848.0,37.2
1,1997-05-01,1997,3033.0,37.6
2,1997-06-01,1997,4958.0,38.2
3,1997-07-01,1997,744.0,37.7
4,1997-08-01,1997,2955.0,37.8
...,...,...,...,...
314,2023-06-01,2023,19762.0,97.3
315,2023-07-01,2023,2764.0,96.4
316,2023-08-01,2023,11093.0,96.5
317,2023-09-01,2023,14642.0,96.4


---

### Visualisation

In [88]:
# Instantiate altair_eco class and apply theme
style = styles.EcoStyles()
style.register_and_enable_theme(dark_mode=True)

### Chart 1. Dual-lines, spending against receipts

In [79]:
alt.Chart(df_dual).mark_line().encode(
    x = alt.X('date:T'),
    y = alt.Y('value:Q'),
    color = alt.Color('series:N')
).transform_filter(
    alt.datum.year > 2021
)

alt.Chart(...)

Notes: would be better if each line was % of GDP

#### Chart 2. Dual Y-axis: Borrowing and Debt % of GDP

In [76]:
# Calculate a 3-month rolling average for borrowing
df['borrowing_3m'] = round(df['borrowing'].rolling(3).mean())
df = df.dropna().reset_index(drop=True)

In [77]:
df

,date,year,borrowing,debt,borrowing_3m
0,1997-08-01,1997,2955.0,37.8,2886.0
1,1997-09-01,1997,2312.0,37.9,2004.0
2,1997-10-01,1997,-1454.0,37.2,1271.0
3,1997-11-01,1997,-1772.0,37.5,-305.0
4,1997-12-01,1997,939.0,37.4,-762.0
...,...,...,...,...,...
310,2023-06-01,2023,19762.0,97.3,18291.0
311,2023-07-01,2023,2764.0,96.4,12731.0
312,2023-08-01,2023,11093.0,96.5,11206.0
313,2023-09-01,2023,14642.0,96.4,9500.0


In [103]:
# Refresh import of eco_styles
importlib.reload(styles)
styles.EcoStyles().register_and_enable_theme(dark_mode=True)

start_date = datetime(2021, 1, 1)
df_temp = df[df['date'] >= start_date].reset_index(drop=True)

base = alt.Chart(df_temp).encode(
    alt.X('date:T').axis(title=None)
)

bar = base.mark_bar(opacity=1).encode(
    alt.Y('borrowing:Q'),
    # Red if positive, blue if negative
    color=alt.condition(
        alt.datum.borrowing > 0,
        alt.value('#36b7b4'),
        alt.value('#eb5c2e')
    )
)

line = base.mark_line().encode(
    alt.Y('debt:Q', scale=alt.Scale(domain=[70, 100])).title('Debt (% of GDP)', titleColor='#5276A7')
)

alt.layer(bar, line).resolve_scale(
    y='independent'
)

# bar.display()

alt.LayerChart(...)

#### Chart 3. Debt % of GDP

In [168]:
# Refresh import of eco_styles
importlib.reload(styles)
styles.EcoStyles().register_and_enable_theme(dark_mode=True)

df_temp = df.copy()

title=alt.Title(
    text='UK Debt',
    subtitle=['Public sector debt, as percent of GDP | Source: ONS via ECO API'],
    dx=32
)

line = alt.Chart(df_temp, title=title).mark_line().encode(
    x=alt.X('date:T'),
    # use Label Expression to add percentage sign to axis
    y=alt.Y('debt:Q', scale=alt.Scale(zero=False, domain=[75, 100]), axis=alt.Axis(labelExpr="datum.value + '%'")),
).transform_filter(
    alt.datum.year >= 2018
)

# Add point and value for latest data
latest_data = df_temp[df_temp['date'] == df_temp['date'].max()].reset_index(drop=True)
latest_data['debt_fmt'] = latest_data['debt'].apply(lambda x: '{:.1%}'.format(x / 100))

point = alt.Chart(latest_data).mark_point().encode(
    x=alt.X('date:T'),
    y=alt.Y('debt:Q')
)

text_value = alt.Chart(latest_data).mark_text(fontSize=13).encode(
    x=alt.X('date:T'),
    y=alt.Y('debt:Q'),
    text=alt.Text('debt_fmt:N')
)

text_month = alt.Chart(latest_data).mark_text(
    fontSize=12,
    dy=-14,
).encode(
    x=alt.X('date:T'),
    y=alt.Y('debt:Q'),
    text=alt.Text('date:T', format='%b %Y')
)

chart = line + point + text_value + text_month

chart.display()

alt.LayerChart(...)

**Save chart**

In [169]:
from datetime import date
# Get today's date in formats YYYYMM & YYYYMMDD
yearmonth = date.today().strftime("%Y%m")
today = date.today().strftime("%Y%m%d")

styles.EcoStyles().save(chart=chart, path=f'../../charts/{yearmonth}', name=f'{today}_UK_Debt_pct_GDP')

**Description**:

Today's ONS data showed £15bn of public sector borrowing in October, leading to a slight increase in the debt-to-GDP ratio, now at 97.8%. How does this compare to last year's trends?


#ChartOfTheDay


Make sure you check out our Data Hub to explore, create and share your own charts #dataviz https://buff.ly/3mkZ2je 
